# KMeans periodically

In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

import glob
import pickle

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.cluster import KMeans, MiniBatchKMeans

from sklearn.metrics import adjusted_mutual_info_score, homogeneity_score, completeness_score, fowlkes_mallows_score    # Supervised / external metrics
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score    # unsupervised / internal metrics

from sklearn.model_selection import TimeSeriesSplit

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Loading

In [2]:
data_root = '/home/c44406/datasets/atb_uofa/loan_portfolio/'

datasets_all = []
for dpath in glob.glob(f'{data_root}datasetv7*.csv'):
    df = pd.read_csv(dpath, dtype={'dunning_level':'object'})
    datasets_all.append(df)

datasets = pd.concat(datasets_all, axis=0, ignore_index=True)

### Cleaning

In [3]:
cols = datasets.columns

# non-feature columns
other_cols = [
    'bus_ptnr_group',
     'cal_day',
     'naics_id',
     'has_loan'
]

used_cols = cols.str.startswith((
    'BRR', 
    'impaired', 
    'Oustanding_principle_on_posting_date', 
    'percentage_rate', 
    'abs_transactions', 
    'n_transactions', 
    'mth_since_brr_update', 
#     'defaults', 
    'transactions', 
    'SUB_SYSTEM', 
    'transaction_type'
))
used_cols = cols[used_cols].to_list()

data = datasets[other_cols + used_cols].copy()
data['cal_day'] = pd.to_datetime(data['cal_day'], errors='coerce')
data = data[data['cal_day'].notnull()].copy()
data.replace(to_replace=[np.inf, -np.inf], value=[np.nan, np.nan], inplace=True)
data = data[data['cal_day'] >= '2008-01-31'].copy()
data = data[data['cal_day'] <= '2021-09-30'].copy()

In [4]:
ppl = Pipeline(steps=[
    ('masking', SimpleImputer(verbose=100)),
    ('scaling', PowerTransformer())
])

transformer = ColumnTransformer(transformers=[
    ('features', ppl, used_cols)
])

data[used_cols] = transformer.fit_transform(data)

In [5]:
data.to_pickle('transformed.pkl')

### 

In [6]:
data = pd.read_pickle('transformed.pkl')
data = data.sort_values('cal_day')
data.head()

,bus_ptnr_group,cal_day,naics_id,has_loan,BRR,impaired,Oustanding_principle_on_posting_date,percentage_rate,transactions,abs_transactions,transactions_db,transactions_cr,n_transactions,SUB_SYSTEM_DP,SUB_SYSTEM_FD,SUB_SYSTEM_IN,SUB_SYSTEM_LN,SUB_SYSTEM_RF,SUB_SYSTEM_RP,SUB_SYSTEM_SP,SUB_SYSTEM_TF,transaction_type_Loan_Disbursement,transaction_type_Payment_Distrib_Loan,transaction_type_Loan_Payment,transaction_type_Bank_Trsf_Deposit_Acct,transaction_type_Installment_Payment,transaction_type_Transfer,transaction_type_Direct_Deposit,transaction_type_Cheque,transaction_type_Deposit_Cheque,transaction_type_Incoming_Wire,transaction_type_Auto_LOC_Repayment,transaction_type_Outgoing_Wire,transaction_type_EFT_Settlement,transaction_type_Direct_Debit,transaction_type_Overdraft_Transfer,transaction_type_Customer_Transfer,transaction_type_LOC_Disburse_RealTime_Adv,transaction_type_Loan_Transfer,transaction_type_EOD_ODP_Trf_Funded_Acc,transaction_type_misc,mth_since_brr_update
793074,11072,2012-01-31,112110.0,True,0.260301,-0.498625,-0.702221,-2.013722,0.002222,1.38552,1.343805,1.387909,0.486283,1.03843,6.361285,5.969544,1.582246,7.945015,7.178748,7.286245,6.994683,3.051386,1.868471,1.759622,3.51824,1.787931,1.555354,1.225621,1.04491,1.194188,4.449774,1.491748,4.536834,2.714883,1.156989,1.566897,6.244328,1.467219,1.487199,1.550769,1.336864,-0.001064
141316,73163,2012-01-31,111999.0,True,0.851234,-0.498625,-0.329923,0.793681,0.002222,1.38552,1.343805,1.387909,0.486283,1.03843,6.361285,5.969544,1.582246,7.945015,7.178748,7.286245,6.994683,3.051386,1.868471,1.759622,3.51824,1.787931,1.555354,1.225621,1.04491,1.194188,4.449774,1.491748,4.536834,2.714883,1.156989,1.566897,6.244328,1.467219,1.487199,1.550769,1.336864,-0.001064
141315,29212,2012-01-31,112110.0,True,-0.376540,-0.498625,-0.306966,0.052506,0.002222,1.38552,1.343805,1.387909,0.486283,1.03843,6.361285,5.969544,1.582246,7.945015,7.178748,7.286245,6.994683,3.051386,1.868471,1.759622,3.51824,1.787931,1.555354,1.225621,1.04491,1.194188,4.449774,1.491748,4.536834,2.714883,1.156989,1.566897,6.244328,1.467219,1.487199,1.550769,1.336864,-16.485819
141314,53335,2012-01-31,111190.0,True,-0.376540,-0.498625,-0.513362,0.262600,0.002222,1.38552,1.343805,1.387909,0.486283,1.03843,6.361285,5.969544,1.582246,7.945015,7.178748,7.286245,6.994683,3.051386,1.868471,1.759622,3.51824,1.787931,1.555354,1.225621,1.04491,1.194188,4.449774,1.491748,4.536834,2.714883,1.156989,1.566897,6.244328,1.467219,1.487199,1.550769,1.336864,-0.001064
141313,96892,2012-01-31,212323.0,True,0.851234,-0.498625,-0.702221,-2.013722,0.002222,1.38552,1.343805,1.387909,0.486283,1.03843,6.361285,5.969544,1.582246,7.945015,7.178748,7.286245,6.994683,3.051386,1.868471,1.759622,3.51824,1.787931,1.555354,1.225621,1.04491,1.194188,4.449774,1.491748,4.536834,2.714883,1.156989,1.566897,6.244328,1.467219,1.487199,1.550769,1.336864,-0.001064


In [7]:
data.tail()

,bus_ptnr_group,cal_day,naics_id,has_loan,BRR,impaired,Oustanding_principle_on_posting_date,percentage_rate,transactions,abs_transactions,transactions_db,transactions_cr,n_transactions,SUB_SYSTEM_DP,SUB_SYSTEM_FD,SUB_SYSTEM_IN,SUB_SYSTEM_LN,SUB_SYSTEM_RF,SUB_SYSTEM_RP,SUB_SYSTEM_SP,SUB_SYSTEM_TF,transaction_type_Loan_Disbursement,transaction_type_Payment_Distrib_Loan,transaction_type_Loan_Payment,transaction_type_Bank_Trsf_Deposit_Acct,transaction_type_Installment_Payment,transaction_type_Transfer,transaction_type_Direct_Deposit,transaction_type_Cheque,transaction_type_Deposit_Cheque,transaction_type_Incoming_Wire,transaction_type_Auto_LOC_Repayment,transaction_type_Outgoing_Wire,transaction_type_EFT_Settlement,transaction_type_Direct_Debit,transaction_type_Overdraft_Transfer,transaction_type_Customer_Transfer,transaction_type_LOC_Disburse_RealTime_Adv,transaction_type_Loan_Transfer,transaction_type_EOD_ODP_Trf_Funded_Acc,transaction_type_misc,mth_since_brr_update
1285766,1907320,2021-09-30,484121.0,True,-1.070290,-0.498625,-0.569075,1.190772,0.005570,-0.813343,-0.603806,-0.879561,-1.192071,-0.677416,-0.157221,-0.167522,-0.020294,-0.125869,-0.139965,-0.137245,-0.143848,-0.327969,0.527556,0.738594,-0.284489,0.637811,-0.6944,-0.911769,-1.221441,0.658289,-0.224742,-0.698203,-0.220423,-0.368787,-1.236195,-0.709261,-0.160268,-0.735859,-0.722711,-0.717396,-0.720807,0.130583
1285767,2070162,2021-09-30,111999.0,True,1.925027,-0.498625,-0.454601,1.108233,0.008087,-0.564106,-0.378442,-0.671811,-0.749181,-1.091792,-0.157221,-0.167522,0.294549,-0.125869,-0.139965,-0.137245,-0.143848,-0.327969,0.846111,0.395989,3.518214,0.914427,-0.6944,-0.911769,-1.221441,-0.943661,-0.224742,-0.698203,-0.220423,-0.368787,-1.236195,-0.709261,-0.160268,-0.735859,-0.722711,-0.717396,-0.907140,0.082184
1285768,37116,2021-09-30,112110.0,True,0.260301,-0.498625,-0.331644,0.461843,0.022504,0.457331,0.476232,0.419475,0.376781,0.329779,-0.157221,-0.167522,0.902987,-0.125869,-0.139965,-0.137245,-0.143848,-0.327969,1.291732,1.344655,-0.284489,1.314236,-0.6944,0.316092,0.295914,1.114549,-0.224742,1.459241,-0.220423,-0.368787,0.406162,1.401983,-0.160268,1.463888,1.484778,1.382532,0.515132,0.029322
1285758,80219,2021-09-30,721114.0,True,2.418340,-0.498625,0.288430,0.368983,0.006152,-0.404081,-0.291363,-0.434079,0.522182,-0.215346,-0.157221,-0.167522,0.036761,-0.125869,-0.139965,-0.137245,-0.143848,-0.327969,0.509636,0.723707,3.518120,0.621674,-0.6944,0.939525,-0.117371,-0.943661,-0.224742,-0.698203,4.536830,-0.368787,0.519106,-0.709261,-0.160268,-0.735859,-0.722711,-0.717396,-0.044280,0.005083
1939307,1913474,2021-09-30,112110.0,True,1.404173,-0.498625,-0.037220,0.052521,0.005116,-0.774394,-0.590874,-0.810833,-0.559918,-0.663188,-0.157221,-0.167522,0.016058,-0.125869,-0.139965,-0.137245,-0.143848,-0.327969,0.531419,0.741799,-0.284489,0.629222,-0.6944,-0.911769,-1.221441,-0.943661,-0.224742,-0.698203,-0.220423,-0.368787,-1.236195,-0.709261,-0.160268,-0.735859,-0.722711,-0.717396,-0.358620,0.020612


In [8]:
loan_data = data[data['has_loan'] == 1]
loan_data = loan_data.drop('has_loan', axis=1).reset_index(drop=True).copy()

other_cols = [
    'bus_ptnr_group',
     'cal_day',
     'naics_id'
]

used_cols = loan_data.columns.difference(other_cols).to_list()
used_cols

['BRR',
 'Oustanding_principle_on_posting_date',
 'SUB_SYSTEM_DP',
 'SUB_SYSTEM_FD',
 'SUB_SYSTEM_IN',
 'SUB_SYSTEM_LN',
 'SUB_SYSTEM_RF',
 'SUB_SYSTEM_RP',
 'SUB_SYSTEM_SP',
 'SUB_SYSTEM_TF',
 'abs_transactions',
 'impaired',
 'mth_since_brr_update',
 'n_transactions',
 'percentage_rate',
 'transaction_type_Auto_LOC_Repayment',
 'transaction_type_Bank_Trsf_Deposit_Acct',
 'transaction_type_Cheque',
 'transaction_type_Customer_Transfer',
 'transaction_type_Deposit_Cheque',
 'transaction_type_Direct_Debit',
 'transaction_type_Direct_Deposit',
 'transaction_type_EFT_Settlement',
 'transaction_type_EOD_ODP_Trf_Funded_Acc',
 'transaction_type_Incoming_Wire',
 'transaction_type_Installment_Payment',
 'transaction_type_LOC_Disburse_RealTime_Adv',
 'transaction_type_Loan_Disbursement',
 'transaction_type_Loan_Payment',
 'transaction_type_Loan_Transfer',
 'transaction_type_Outgoing_Wire',
 'transaction_type_Overdraft_Transfer',
 'transaction_type_Payment_Distrib_Loan',
 'transaction_type_Trans

## Preprocessing

In [9]:
pre_ppl = Pipeline(steps=[
    ('masking', SimpleImputer(strategy='constant', fill_value=0.0))
])

pre_transformer = ColumnTransformer(transformers=[
    ('numeric', pre_ppl, used_cols)
])


loan_data[used_cols] = pre_transformer.fit_transform(loan_data)

In [10]:
loan_data.head()

,bus_ptnr_group,cal_day,naics_id,BRR,impaired,Oustanding_principle_on_posting_date,percentage_rate,transactions,abs_transactions,transactions_db,transactions_cr,n_transactions,SUB_SYSTEM_DP,SUB_SYSTEM_FD,SUB_SYSTEM_IN,SUB_SYSTEM_LN,SUB_SYSTEM_RF,SUB_SYSTEM_RP,SUB_SYSTEM_SP,SUB_SYSTEM_TF,transaction_type_Loan_Disbursement,transaction_type_Payment_Distrib_Loan,transaction_type_Loan_Payment,transaction_type_Bank_Trsf_Deposit_Acct,transaction_type_Installment_Payment,transaction_type_Transfer,transaction_type_Direct_Deposit,transaction_type_Cheque,transaction_type_Deposit_Cheque,transaction_type_Incoming_Wire,transaction_type_Auto_LOC_Repayment,transaction_type_Outgoing_Wire,transaction_type_EFT_Settlement,transaction_type_Direct_Debit,transaction_type_Overdraft_Transfer,transaction_type_Customer_Transfer,transaction_type_LOC_Disburse_RealTime_Adv,transaction_type_Loan_Transfer,transaction_type_EOD_ODP_Trf_Funded_Acc,transaction_type_misc,mth_since_brr_update
0,11072,2012-01-31,112110.0,0.260301,-0.498625,-0.702221,-2.013722,0.002222,1.38552,1.343805,1.387909,0.486283,1.03843,6.361285,5.969544,1.582246,7.945015,7.178748,7.286245,6.994683,3.051386,1.868471,1.759622,3.51824,1.787931,1.555354,1.225621,1.04491,1.194188,4.449774,1.491748,4.536834,2.714883,1.156989,1.566897,6.244328,1.467219,1.487199,1.550769,1.336864,-0.001064
1,73163,2012-01-31,111999.0,0.851234,-0.498625,-0.329923,0.793681,0.002222,1.38552,1.343805,1.387909,0.486283,1.03843,6.361285,5.969544,1.582246,7.945015,7.178748,7.286245,6.994683,3.051386,1.868471,1.759622,3.51824,1.787931,1.555354,1.225621,1.04491,1.194188,4.449774,1.491748,4.536834,2.714883,1.156989,1.566897,6.244328,1.467219,1.487199,1.550769,1.336864,-0.001064
2,29212,2012-01-31,112110.0,-0.376540,-0.498625,-0.306966,0.052506,0.002222,1.38552,1.343805,1.387909,0.486283,1.03843,6.361285,5.969544,1.582246,7.945015,7.178748,7.286245,6.994683,3.051386,1.868471,1.759622,3.51824,1.787931,1.555354,1.225621,1.04491,1.194188,4.449774,1.491748,4.536834,2.714883,1.156989,1.566897,6.244328,1.467219,1.487199,1.550769,1.336864,-16.485819
3,53335,2012-01-31,111190.0,-0.376540,-0.498625,-0.513362,0.262600,0.002222,1.38552,1.343805,1.387909,0.486283,1.03843,6.361285,5.969544,1.582246,7.945015,7.178748,7.286245,6.994683,3.051386,1.868471,1.759622,3.51824,1.787931,1.555354,1.225621,1.04491,1.194188,4.449774,1.491748,4.536834,2.714883,1.156989,1.566897,6.244328,1.467219,1.487199,1.550769,1.336864,-0.001064
4,96892,2012-01-31,212323.0,0.851234,-0.498625,-0.702221,-2.013722,0.002222,1.38552,1.343805,1.387909,0.486283,1.03843,6.361285,5.969544,1.582246,7.945015,7.178748,7.286245,6.994683,3.051386,1.868471,1.759622,3.51824,1.787931,1.555354,1.225621,1.04491,1.194188,4.449774,1.491748,4.536834,2.714883,1.156989,1.566897,6.244328,1.467219,1.487199,1.550769,1.336864,-0.001064


## Getting period-data

In [11]:
def generate_datasets(data, col, start_date, end_date, train_size, test_size):
    data = data[data[col] >= start_date]
    data = data[data[col] <= end_date]
    
    datasets = []
    training_split_dates = pd.date_range(start_date, end_date, freq=f'{train_size}M')[1:]
    testing_split_dates = training_split_dates.shift(test_size, freq='M')
    for i in range(len(training_split_dates)):
        date1 = training_split_dates[i]
        date2 = testing_split_dates[i]
        
        train_data = data[data[col] < date1].copy()
        test_data = data[data[col] >= date1].copy()
        test_data = test_data[test_data[col]< date2]

        datasets.append((train_data, test_data))
    
    return datasets

In [26]:
start_date = '2016-01-31'
end_date = '2020-12-31'
training_months = 19
testing_months = 3

datasets = generate_datasets(loan_data, 'cal_day', start_date, end_date, training_months, testing_months)

In [27]:
print(datasets[0][0]['cal_day'].unique())
print(datasets[0][1]['cal_day'].unique())
print()
print(datasets[-1][0]['cal_day'].unique())
print(datasets[-1][1]['cal_day'].unique())

['2016-01-31T00:00:00.000000000' '2016-02-29T00:00:00.000000000'
 '2016-03-31T00:00:00.000000000' '2016-04-30T00:00:00.000000000'
 '2016-05-31T00:00:00.000000000' '2016-06-30T00:00:00.000000000'
 '2016-07-31T00:00:00.000000000' '2016-08-31T00:00:00.000000000'
 '2016-09-30T00:00:00.000000000' '2016-10-31T00:00:00.000000000'
 '2016-11-30T00:00:00.000000000' '2016-12-31T00:00:00.000000000'
 '2017-01-31T00:00:00.000000000' '2017-02-28T00:00:00.000000000'
 '2017-03-31T00:00:00.000000000' '2017-04-30T00:00:00.000000000'
 '2017-05-31T00:00:00.000000000' '2017-06-30T00:00:00.000000000'
 '2017-07-31T00:00:00.000000000']
['2017-08-31T00:00:00.000000000' '2017-09-30T00:00:00.000000000'
 '2017-10-31T00:00:00.000000000']

['2016-01-31T00:00:00.000000000' '2016-02-29T00:00:00.000000000'
 '2016-03-31T00:00:00.000000000' '2016-04-30T00:00:00.000000000'
 '2016-05-31T00:00:00.000000000' '2016-06-30T00:00:00.000000000'
 '2016-07-31T00:00:00.000000000' '2016-08-31T00:00:00.000000000'
 '2016-09-30T00:00:00

## KMeans

### PCA

In [28]:
len(used_cols)

38

In [29]:
reduced_datasets = []
for train, test in datasets:
    pca = PCA(n_components=35, svd_solver='randomized', iterated_power='auto')
    reduced_train_data = pca.fit_transform(train[used_cols].to_numpy())
    reduced_test_data = pca.fit_transform(test[used_cols].to_numpy())
    reduced_datasets.append((reduced_train_data, reduced_test_data))


### 

In [30]:
models = []
for train, _ in reduced_datasets:
    kmeans = KMeans(n_clusters=10)
    kmeans.fit(train)
    models.append(kmeans)


In [31]:
# getting some metrics of kmeans for each reduced test data
scores = []
for i in range(len(models)):
    model = models[i]
    test = reduced_datasets[i][1]
    labels_pred = model.predict(test)
    score = davies_bouldin_score(test, labels_pred)
    scores.append(score)
    
print(np.mean(scores))

2.2415949966338515


In [32]:
# Refit KMeans with previous cluster centroids (training data)
consist_scores = []

for i in range(len(models)):
    train = reduced_datasets[i][0]
    model = models[i]
    labels_pred = model.labels_
    centroids = model.cluster_centers_
    for _ in range(10):
        kmeans_ini = KMeans(n_clusters=num_clusters, init=centroids, n_init=1)
        kmeans_ini.fit(train)
        consist_scores.append(adjusted_mutual_info_score(labels_pred, kmeans_ini.labels_))
        
        labels_pred = kmeans_ini.labels_
        centroids = kmeans_ini.cluster_centers_
    
print(np.mean(consist_scores))

0.9985308354464943


### MiniBatch

In [33]:
seq_models = []
for train, _ in reduced_datasets:
    seq_kmeans = MiniBatchKMeans(n_clusters=10, batch_size=512)
    seq_kmeans.fit(train)
    seq_models.append(seq_kmeans)


In [34]:
# getting some metrics of kmeans for each reduced test data
scores = []
for i in range(len(seq_models)):
    model = seq_models[i]
    test = reduced_datasets[i][1]
    labels_pred = model.predict(test)
    score = davies_bouldin_score(test, labels_pred)
    scores.append(score)
    
print(np.mean(scores))

2.7037679032972677


In [35]:
# Refit MiniBatchKMeans with partial fit (training data)
consist_scores = []

for i in range(len(seq_models)):
    train = reduced_datasets[i][0]
    model = seq_models[i]
    labels_pred = model.labels_
    for _ in range(10):
        model.partial_fit(train)
        consist_scores.append(adjusted_mutual_info_score(labels_pred, model.labels_))
        labels_pred = model.labels_
    
print(np.mean(consist_scores))

0.9810983242000761
